# Задание 1

Задача: Создать чат бота для получения информации об исследованиях космоса

Описание: Создайте комплексное приложение командной строки, которое будет использоваться в качестве панели управления исследованиями космоса. Данное приложение будет обращаться к https://api.nasa.gov/ для предоставления пользователям набора информации о космосе, включая:

- Астрономическая картинка дня (APOD): Отображение APOD с пояснениями к нему.
- Фотографии с марсохода: позволяет пользователям выбирать и фильтровать фотографии с марсохода по дате и типу камеры.
- Объекты, сближающиеся с Землей (ОСЗ): Поиск и отображение информации об объекте, сближающихся с Землей, на определенную дату, включая их размеры и потенциальную опасность.
- Данные о космической погоде: Отображают последние данные о космической погоде, включая солнечные вспышки и геомагнитные бури.
Приложение должно позволять пользователям ориентироваться в этих функциях, корректно обрабатывать ошибки и обеспечивать удобство работы.

Требования:
- Пользовательский ввод: Приложение должно предложить пользователю ввести данные, чтобы выбрать, какую функцию он хочет изучить.
- Проверка данных: Убедитесь, что пользовательские данные (например, даты) проверены.
- Обработка ошибок: Корректно обрабатывайте ошибки API и неверные ответы.
- Представление данных: Представляйте данные в четкой и организованной форме.
- Опция выхода: позволяет пользователям выходить из приложения в любое время.

In [ ]:
pip install requests

In [ ]:
import requests


def start():
  num = input("""
  1 -> Астрономическая картинка дня (APOD) \n
  2 -> Фотографии с марсохода \n
  3 -> Объекты, сближающиеся с Землей (ОСЗ) \n
  4 -> Данные о космической погоде \n
  5 -> Выход \n
  """)
  numLogger(int(num))


def numLogger(num):
  if num == 1:
    apod()
  elif num == 2:
    marsRover()
  elif num == 3:
    neoWS()
  elif num == 4:
    cosmicWeather()
  elif num == 5:
    return 1
  else:
    print('Вы ввели некорректный номер.')
  start()


def apod():
  response = requests.get('https://api.nasa.gov/planetary/apod?api_key=zIjRwfQqJF9Waj7xIkxee6x752q1nMiSmlydOLwd')
  if response.status_code == 200 or response.status_code == 201:
    json = response.json()
    expl = ''
    url = ''
    for k, v in json.items():
      if k == 'explanation':
        expl = v
      if k == 'url':
        url = v
    print(f'\n Изображение дня: {url} \n Пояснение: {expl}')
  else:
    print('Ошибка API')


def marsRover():
  date = input('\nВведите дату без нулей в формате ГОД-МЕСЯЦ-ЧИСЛО:\n')
  response = requests.get(f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?earth_date={date}&api_key=zIjRwfQqJF9Waj7xIkxee6x752q1nMiSmlydOLwd')
  if response.status_code == 200 or response.status_code == 201:
    camera_types = []
    json = response.json()
    for v in json['photos']:
      for ke, val in v['camera'].items():
        if ke == 'name':
          if val not in camera_types:
            camera_types.append(val)
    user_type = input(f'\nВыберите тип камеры из списка:\n{camera_types}\n')
    for v in json['photos']:
      if user_type in v['camera'].values():
        print()
        print('Изображение по заданным параметрам: ', v['img_src'])
  else:
    print('Ошибка API')

def neoWS():
   date = input('\nВведите дату без нулей в формате ГОД-МЕСЯЦ-ЧИСЛО:\n')
   response = requests.get(f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&api_key=zIjRwfQqJF9Waj7xIkxee6x752q1nMiSmlydOLwd')
   if response.status_code == 200 or response.status_code == 201:
    json = response.json()
    name = ''
    danger = ''
    estimated_diam = 0
    for k, v in json.items():
      if k == 'near_earth_objects':
        for val in v.values():
          for value in val:
            for ke, va in value.items():
              if ke == 'name':
                name = va
              if ke == 'estimated_diameter':
                min_diam = 0
                max_diam = 0
                for key, valu in value[ke].items():
                  if key == 'meters':
                    for keyy, valuee in valu.items():
                      if keyy == 'estimated_diameter_min':
                        min_diam = valuee
                      if keyy == 'estimated_diameter_max':
                        max_diam = valuee
              if ke == 'is_potentially_hazardous_asteroid':
                if va:
                  danger = 'Да'
                else:
                  danger = 'Нет'
                print(f"""
                Имя: {name},
                Предпологаемый диаметр: {(max_diam + min_diam) / 2},
                Опасен ли он для земли: {danger}
                      """)
   else:
    print('Ошибка API')


def cosmicWeather():
  response = requests.get('https://api.nasa.gov/DONKI/FLR?api_key=zIjRwfQqJF9Waj7xIkxee6x752q1nMiSmlydOLwd')
  json = response.json()
  if response.status_code == 200 or response.status_code == 201:
    print('\n Солнечные вспышки: \n')
    for flare in json:
      for k, v in flare.items():
        if k == 'flrID':
          print('Имя: ', v)
        if k == 'beginTime':
          print('Время начала: ', v)
        if k == 'peakTime':
          print('Время пика: ', v)
        if k == 'endTime':
          print('Время конца: ', v, '\n')
  else:
    print('Ошибка API')

  response = requests.get('https://api.nasa.gov/DONKI/GST?api_key=zIjRwfQqJF9Waj7xIkxee6x752q1nMiSmlydOLwd')
  json = response.json()
  if response.status_code == 200 or response.status_code == 201:
    print('\n Геомагнитные бури \n')
    for storm in json:
      for k, v in storm.items():
        if k == 'gstID':
            print('Имя: ', v)
        if k == 'startTime':
          print('Время начала: ', v, '\n')
  else:
    print('Ошибка API')


print('Чат бот для получения информации об исследованиях космоса')
start()




Чат бот для получения информации об исследованиях космоса

  1 -> Астрономическая картинка дня (APOD) 

  2 -> Фотографии с марсохода 
 
  3 -> Объекты, сближающиеся с Землей (ОСЗ) 

  4 -> Данные о космической погоде 

  5 -> Выход 

  4

 Солнечные вспышки: 

Имя:  2024-10-06T00:09:00-FLR-001
Время начала:  2024-10-06T00:09Z
Время пика:  2024-10-06T00:18Z
Время конца:  2024-10-06T00:29Z 

Имя:  2024-10-06T01:30:00-FLR-001
Время начала:  2024-10-06T01:30Z
Время пика:  2024-10-06T01:47Z
Время конца:  2024-10-06T02:04Z 

Имя:  2024-10-06T15:14:00-FLR-001
Время начала:  2024-10-06T15:14Z
Время пика:  2024-10-06T15:21Z
Время конца:  2024-10-06T15:25Z 

Имя:  2024-10-06T16:41:00-FLR-001
Время начала:  2024-10-06T16:41Z
Время пика:  2024-10-06T16:45Z
Время конца:  2024-10-06T16:51Z 

Имя:  2024-10-06T18:52:00-FLR-001
Время начала:  2024-10-06T18:52Z
Время пика:  2024-10-06T18:59Z
Время конца:  2024-10-06T19:04Z 

Имя:  2024-10-06T20:52:00-FLR-001
Время начала:  2024-10-06T20:52Z
Время пика:

# Задание 2

Описание задачи

Цель этой задачи - создать скрипт на Python, который взаимодействует с API Чикагского института искусств (https://api.artic.edu/docs/) для извлечения и отображения произведений искусства. Скрипт должен позволять пользователям просматривать работы по страницам, фильтровать их по имени художника и просматривать подробную информацию о выбранных произведениях искусства. Ниже приведены требования и функциональные возможности, которые необходимо реализовать:

Требования:
Извлекать произведения искусства:

- Создайте функцию, которая извлекает список произведений искусства из API Чикагского института искусств.
Функция должна принимать параметр page для разбивки на страницы и возвращать список произведений искусства вместе с информацией о разбивке на страницы.
Фильтровать произведения искусства:

- Реализуйте функцию, которая фильтрует список произведений искусства на основе имени указанного художника. Функция должна возвращать список работ, которые соответствуют имени художника (без учета регистра).
Отображать подробную информацию об оформлении:

- Напишите функцию, которая отображает названия работ для пользователя и позволяет ему выбрать одну из них, введя соответствующий номер.
После выбора функция должна отображать подробную информацию о выбранном произведении, включая название, исполнителя, дату и носитель.
Разбивка на страницы и взаимодействие с пользователем:

- Создайте основную функцию, которая управляет выборкой произведений и взаимодействием с пользователем.

Разрешите пользователям перемещаться по страницам с произведениями искусства, выполнять фильтрацию по исполнителю или выходить из программы.

Если страниц с произведениями искусства несколько, укажите варианты перехода к следующей странице, предыдущей странице, фильтрации по исполнителю или выхода из программы.

In [ ]:
import requests

def start():
  num = input("""
  1 -> Извлекать произведения искусства \n
  2 -> Фильтровать произведения искусства \n
  3 -> Выход \n
  """)
  numLogger(int(num))


def numLogger(num):
  if num == 1:
    page = input('Введите номер страницы: ')
    getArtworks(page)
  elif num == 2:
    name = input('Введите имя художника: ')
    filterArtworks(name)
  elif num == 3:
    return 1
  else:
    print('Вы ввели некорректный номер.')
  start()


def getArtworks(page):
  response = requests.get(f'https://api.artic.edu/api/v1/artworks?page={page}')
  if response.status_code == 200 or response.status_code == 201:
    json = response.json()
    for v in json['data']:
      print(v['id'], v['title'])
  else:
    print('Ошибка API')


def filterArtworks(name):
  response = requests.get(f'https://api.artic.edu/api/v1/artworks?q={name}')
  if response.status_code == 200 or response.status_code == 201:
    json = response.json()
    for v in json['data']:
      print(v['id'], v['title'])
  else:
    print('Ошибка API')


def chooseArtwork():
  num = input('Введите номер произведения: ')
  response = requests.get(f'https://api.artic.edu/api/v1/artworks/{num}')
  if response.status_code == 200 or response.status_code == 201:
    json = response.json()
    for k, v in json['data'].items():
      if k == 'title':
        print(v)
      if k == 'artist_title':
        print(v)
      if k == 'date_display':
        print(v)


start()


  1 -> Извлекать произведения искусства 

  2 -> Фильтровать произведения искусства 

  3 -> Выход 

  3


# Задание 3

Задача: Создать программу по управлению портфелем криптовалют

Цель: Создать скрипт на Python, который извлекает цены на криптовалюты в режиме реального времени, позволяет пользователям управлять портфелем криптовалют, вычисляет общую стоимость портфеля, отслеживает изменения цен и предоставляет исторические данные о ценах для анализа.

Требования:
Получение текущих цен на криптовалюты:

Используйте https://docs.coingecko.com/ для получения актуальных цен на список криптовалют.

Управление портфелем:

- Позволяет пользователю создавать портфель криптовалют и управлять им, указывая количество каждой криптовалюты, которой он владеет.
- Расчитывает общую стоимость портфеля в указанной фиатной валюте (например, долларах США).

Отслеживание изменения цен:

- Отображение процентного изменения цены для каждой криптовалюты в портфеле за последние 24 часа.
- Выделите все криптовалюты, стоимость которых значительно увеличилась или снизилась.

Поиск исторических данных о ценах:

- Получение исторических данных о ценах на указанную криптовалюту за последнюю неделю.
- Предоставьте пользователю возможность визуализировать эти данные в простом текстовом формате (например, цены за день).

Взаимодействие с пользователем:

- Реализуйте интерфейс командной строки для ввода данных пользователем.
- Предоставьте опции для получения текущих цен, управления портфелем, просмотра изменений цен или анализа исторических данных.


In [ ]:
import requests


def start():
    print("""
    1 -> Добавить криптовалюту
    2 -> Удалить криптовалюту
    3 -> Просмотреть изменения в портфеле
    4 -> Исторические данные о криптовалюте
    5 -> Общая стоимость портфеля
    6 -> Выход
    """)
    num = int(input("Введите номер действия: "))
    action(num)


def action(num):
    if num == 1:
        crypto = input("Введите имя криптовалюты: ").strip().lower()
        amount = float(input("Введите количество: "))
        add_crypto(crypto, amount)
    elif num == 2:
        crypto = input("Введите имя криптовалюты: ").strip().lower()
        amount = float(input("Введите количество: "))
        remove_crypto(crypto, amount)
    elif num == 3:
        view_portfolio()
    elif num == 4:
        crypto = input("Введите имя криптовалюты: ").strip().lower()
        show_historical_data(crypto)
    elif num == 5:
        show_total_value()
    elif num == 6:
        print("Выход.")
        return
    else:
        print("Некорректный ввод. Попробуйте снова.")
    start()

holdings = {}


def add_crypto(crypto, amount):
    if crypto in holdings:
        holdings[crypto] += amount
    else:
        holdings[crypto] = amount
    print(f"Добавлено {amount} {crypto} в портфель.")


def remove_crypto(crypto, amount):
    if crypto in holdings:
        holdings[crypto] = max(0, holdings[crypto] - amount)
        if holdings[crypto] == 0:
            del holdings[crypto]
        print(f"Удалено {amount} {crypto} из портфеля.")
    else:
        print("Криптовалюта не найдена в портфеле.")


def get_current_price(crypto_list, currency="usd"):
    ids = ",".join(crypto_list)
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={ids}&vs_currencies={currency}&include_24hr_change=true"
    response = requests.get(url)
    return response.json()


def view_portfolio():
    crypto_list = list(holdings.keys())
    prices = get_current_price(crypto_list)

    print("\nИзменения в портфеле:")
    for crypto, amount in holdings.items():
        if crypto in prices:
            price = prices[crypto]["usd"]
            change = prices[crypto]["usd_24h_change"]
            print(f"{crypto.capitalize()}: {amount} шт., Текущая цена: ${price:.2f}, Изменение за 24ч: {change:.2f}%")
    print()

def get_historical_data(crypto_id, days=7):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart?vs_currency=usd&days={days}"
    response = requests.get(url)
    return [(d[0], d[1]) for d in response.json().get("prices", [])]


def show_historical_data(crypto):
    data = get_historical_data(crypto)
    print(f"Исторические данные для {crypto} за последние 7 дней:")
    for timestamp, price in data:
        print(f"Дата: {timestamp}, Цена: {price}")


def show_total_value():
    crypto_list = list(holdings.keys())
    prices = get_current_price(crypto_list)
    total = 0
    for crypto, amount in holdings.items():
        if crypto in prices:
            total += prices[crypto]["usd"] * amount
    print(f"Общая стоимость портфеля: ${total:.2f}")


start()



    1 -> Добавить криптовалюту
    2 -> Удалить криптовалюту
    3 -> Просмотреть изменения в портфеле
    4 -> Исторические данные о криптовалюте
    5 -> Общая стоимость портфеля
    6 -> Выход
    
Введите номер действия: 1
Введите имя криптовалюты: bitcoin
Введите количество: 1
Добавлено 1.0 bitcoin в портфель.

    1 -> Добавить криптовалюту
    2 -> Удалить криптовалюту
    3 -> Просмотреть изменения в портфеле
    4 -> Исторические данные о криптовалюте
    5 -> Общая стоимость портфеля
    6 -> Выход
    
Введите номер действия: 5
Общая стоимость портфеля: $76363.00

    1 -> Добавить криптовалюту
    2 -> Удалить криптовалюту
    3 -> Просмотреть изменения в портфеле
    4 -> Исторические данные о криптовалюте
    5 -> Общая стоимость портфеля
    6 -> Выход
    
Введите номер действия: 6
Выход.


# Дополнительно: Задание 4

Задание 4: Проектное

Вам необходимо самостоятельно найти откртое API предоставляющее информацию в открытом доступе и реализовать собственный проект!


Критерии приемки результата:

- Проект включает в себя не менее 5 возможостей для пользователя
- Проект позволяет использовать все возможности проекта пользователю при помощи взаимодействия через коммандную строку
- Проект работает с открытым API (это значит что при проверке вашей работы преподавателем, преподавателю необходимо просто запустить ячейку с кодом вашего проекта и она будет работать без дополнительных манипуляции)
- Проект должен обязательно включать в себя ряд используемых конструкции:
    - Функции
    - Условные конструкции
    - Ввод/вывод
    - Словари/Списки
- Допускается использование библиотек:
    - requests
    - datetime
    - random

**Здесь добавьте описание вашего проекта**

In [ ]:
#  А здесь код